In [1]:
from keras.layers import Dropout
from keras import regularizers

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop
import cv2
import keras.backend as K
from keras.models import Model
import pandas as pd
from keras.applications import ResNet50
base_model = ResNet50(weights = None, include_top = False, input_shape= (240,320,1))
x = base_model.output
x = Flatten()(x)
predictions = Dense(4)(x)
model = Model(inputs = base_model.input, output = predictions)
model.summary()
import os
SAVE_WEIGHTS_CHECKPOINT_FILEPATH = os.path.join("model_new_resnet_50_best_new_aug_half.weights")
SAVE_PREDICTIONS = True
SAVE_PREDICTIONS_DIR = os.path.join("predictions")
MODEL_IMAGE_HEIGHT = 480
MODEL_IMAGE_WIDTH = 640
def iou_metric(y_true, y_pred):
    # iou as metric for bounding box regression
    # input must be as [x1, y1, x2-x1, y2-y1]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] + 1) * K.abs(K.transpose(y_true)[3] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] + 1) * K.abs(K.transpose(y_pred)[3] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2] + K.transpose(y_true)[0] , K.transpose(y_pred)[2] + K.transpose(y_pred)[0])
    overlap_3 = K.minimum(K.transpose(y_true)[3] + K.transpose(y_true)[1] , K.transpose(y_pred)[3] + K.transpose(y_pred)[1])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())
    return -iou
# compile with mean squared error
print("Compiling...")
model.compile(loss='mean_squared_error', optimizer=RMSprop(lr = 0.0001), metrics=[iou_metric])

Using TensorFlow backend.
C:\Users\Konsang\Anaconda3\envs\py36\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\Konsang\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 320, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 326, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 120, 160, 64) 3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 120, 160, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [2]:
train_names_cv = ['traincv_augs/' + img for img in os.listdir('traincv_augs/')]
val_names_cv = ['valcv_augs/' + img for img in os.listdir('valcv_augs/')]
checkpoint = ModelCheckpoint(SAVE_WEIGHTS_CHECKPOINT_FILEPATH, monitor= 'val_loss', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_iou_metric', patience=15)
train_cv = pd.read_csv('traincv_augs.csv')
val_cv = pd.read_csv('valcv_augs.csv')
from skimage.io import imread
def get_input(path):
    im = cv2.imread(path)
    im = cv2.resize(im, (320,240))
    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    #ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return np.expand_dims(gray, axis = -1)
import numpy as np
import pandas as pd
def get_output(path,train_cv):
    targetSize_y = 240
    targetSize_x = 320
    x_scale = targetSize_x / 640
    y_scale = targetSize_y / 480
    img_id = path.split('/')[-1]
    x1 = train_cv.loc[train_cv['image_name'] == img_id,'x1'].values[0]
    x2 = train_cv.loc[train_cv['image_name'] == img_id,'x2'].values[0]
    y1 = train_cv.loc[train_cv['image_name'] == img_id,'y1'].values[0]
    y2 = train_cv.loc[train_cv['image_name'] == img_id,'y2'].values[0]
    x1 = int(np.round(x1 * x_scale))
    x2 = int(np.round(x2 * x_scale))
    y1 = int(np.round(y1 * y_scale))
    y2 = int(np.round(y2* y_scale))
    labels = [x1,y1,x2-x1,y2-y1]
    return labels
def image_generator(files,train_cv,batch_size = 2):
    while True:
          # Select files (paths/indices) for the batch
          batch_paths = np.random.choice(a = files, 
                                         size = batch_size)
          batch_input = []
          batch_output = [] 
        
          # Read in each input, perform preprocessing and get labels
          for input_path in batch_paths:
              inputo = get_input(input_path)
              if inputo is None:
                continue
              output = get_output(input_path, train_cv)
              batch_input += [inputo]
              batch_output += [output]
          # Return a tuple of (input,output) to feed the network
          batch_x = np.array(batch_input)
          batch_y = np.array(batch_output)
          yield(batch_x, batch_y)
train_cv = pd.read_csv('traincv_augs.csv')
val_cv = pd.read_csv('valcv_augs.csv')
batch_sizex = 8

In [3]:
model.load_weights('model_new_resnet_50_best_new_aug_half.weights')

In [4]:
model.fit_generator(image_generator(train_names_cv,train_cv, batch_size=batch_sizex), validation_data=image_generator(val_names_cv, val_cv, batch_size=batch_sizex), steps_per_epoch=len(train_cv)//batch_sizex, validation_steps=len(val_cv)//batch_sizex, epochs= 100, callbacks = [checkpoint, earlystopping])

Epoch 1/100
4801/4801 [==============================] - 885s 184ms/step - loss: 153.0085 - iou_metric: -0.9072 - val_loss: 311.7730 - val_iou_metric: -0.8791

Epoch 00001: val_loss improved from inf to 311.77297, saving model to model_new_resnet_50_best_new_aug_half.weights
Epoch 2/100
4801/4801 [==============================] - 859s 179ms/step - loss: 108.4853 - iou_metric: -0.9093 - val_loss: 203.8469 - val_iou_metric: -0.8797

Epoch 00002: val_loss improved from 311.77297 to 203.84690, saving model to model_new_resnet_50_best_new_aug_half.weights
Epoch 3/100
4801/4801 [==============================] - 867s 180ms/step - loss: 101.4112 - iou_metric: -0.9126 - val_loss: 288.2544 - val_iou_metric: -0.8782

Epoch 00003: val_loss did not improve from 203.84690
Epoch 4/100
 398/4801 [=>............................] - ETA: 11:39 - loss: 37.2469 - iou_metric: -0.9133

KeyboardInterrupt: 

In [5]:
model.save_weights('overtraindresnet.weights')